In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.distributions.categorical import Categorical
from itertools import combinations


a = 0.9 #probability that user picks a recommendation, given that he continues
q = 0.1 #probability that the user exits (terminates) the session

K = 50 #num of videos in platform (represents the states - numbered from 0 to K-1)
N = 2  #num of recommendations
u_min = 0.1 #relevance threshold


U = np.random.rand(K, K) #relevance matrix
np.fill_diagonal(U, 0)
U_bool = U > u_min #0 if j is irrelevant to i, 1 if j is relevant to i
C = np.random.choice([0, 1], size=K, p=[0.8, 0.2])

# for testing purposes K = 20
# U = np.random.uniform(0.001, 1, size=(K, K))
# np.fill_diagonal(U, 0)
# U_bool = U > u_min #0 if j is irrelevant to i, 1 if j is relevant to i
# C = np.array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0 ,0 ,0, 0, 0, 0, 0, 0])


hit_table = torch.tensor(U_bool & C).float()
cached_tensor = torch.tensor(C).float()


actions = list(combinations(range(K), N))
num_of_actions = len(actions)



##Neural Net
def mlp(num_of_actions):
   return nn.Sequential(
            nn.Linear(K, 2*K),
            nn.ReLU(),
            nn.Linear(2*K, num_of_actions),
            nn.Identity()
   )


def train(epochs=50, batch_size=1000, learning_rate=0.01, eval=False, eval_episodes=1000):


    logits_net = mlp(num_of_actions) #policy net
    v_approx_net = nn.Sequential(    #value function approximation net
            nn.Linear(K, K),
            nn.ReLU(),
            nn.Linear(K, 1),
            nn.ReLU()
    )

    def get_policy(obs):


        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    def get_action(obs):
        return get_policy(obs).sample().item()


    # make loss function whose gradient, for the right data, is policy gradient
    def compute_loss(obs, act, weights):
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()

    def compute_v_loss(pred, target):
      se = (pred - target) ** 2
      mse = se.mean()
      return mse


    def reward_to_go(rews):
      n = len(rews)
      rtgs = np.zeros_like(rews)
      for i in reversed(range(n)):
          rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
      return rtgs


    def evaluate_net():
      total_reward = 0
      for i in range(eval_episodes):
        state = reset()
        done = False
        while not done:

          #Case 1: Input is relevance row
          obs = torch.tensor(U[state], dtype=torch.float64).float()
          #Case 2: Input is the concatenation of cached and U[state]
          # obs = torch.cat((torch.as_tensor(U[state]), cached_tensor), dim=0).float()

          act = get_action(obs)
          state, rew, done = env(state, actions[act])

          total_reward += rew

      print(f'Policy network achieved: {total_reward / eval_episodes}')


      return

    v_optimizer = Adam(v_approx_net.parameters(), lr=learning_rate)
    optimizer = Adam(logits_net.parameters(), lr=learning_rate)

    pg_epoch_avg_ret = []

    def train_one_epoch(eval=False):
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths
        batch_rews_to_go = []   #for G_t at each timestep at each episode


        # reset episode-specific variables
        state = reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep
        ep_v_s = []             #list with the value function estimations of this episode
        episode_num = 0         #keep track of how many episodes has been played


        # collect experience by acting in the environment with current policy
        while True:

            # save obs
            #Case 1: Input is relevance row
            obs = torch.tensor(U[state], dtype=torch.float64).float()
            #Case 2: Input is the concatenation of cached and U[state]
            # obs = torch.cat((torch.as_tensor(U[state]), cached_tensor), dim=0).float()

            batch_obs.append(obs.tolist().copy())

            # act in the environment
            act = get_action(obs)

            v_s = v_approx_net(obs)

            state, rew, done = env(state, actions[act]) #obs is one input vector for the neural net. It is the state.

            # save action, reward, and estimation
            batch_acts.append(act)
            ep_rews.append(rew)
            ep_v_s.append(v_s.item())


            if done:

                episode_num += 1
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)

                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)


                rew_to_go = reward_to_go(ep_rews)
                advantage = rew_to_go - np.array(ep_v_s)

                batch_rews_to_go += list(rew_to_go) #will be used for fitting baseline

                # # the weight for each logprob(a_t|s_t) is reward-to-go from t
                # batch_weights += list(reward_to_go(ep_rews))

                #the weight is the advantage estimation
                batch_weights += list(advantage)

                # reset episode-specific variables
                obs, done, ep_rews, v_s, ep_v_s = reset(), False, [], [], []


                # end experience loop if we have enough of it
                if episode_num == batch_size:
                    break




        #one step value function approximation network update
        v_optimizer.zero_grad()
        batch_estimates = v_approx_net(torch.as_tensor(batch_obs, dtype=torch.float32))
        criterion = nn.MSELoss()
        v_loss = criterion(batch_estimates, torch.as_tensor(batch_rews_to_go, dtype=torch.float32).unsqueeze(1))
        v_loss.backward()
        v_optimizer.step()

        # take a single policy gradient update step
        optimizer.zero_grad()
        batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
        batch_loss.backward()
        optimizer.step()
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        pg_epoch_avg_ret.append(np.mean(batch_rets))
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))


    plot_curves(pg_epoch_avg_ret)

    # evaluate
    if eval: evaluate_net()


    return pg_epoch_avg_ret


##########Q LEARNING#############
def q_learning(gamma = 1, alpha=0.1, e_t=0.1, epochs=20, batch_size=5000,eval=True, eval_episodes=1000):

    def get_action(Q):


      return actions[a]

    def eval_q_learning(pi):
      total_reward = 0
      for i in range(eval_episodes):
        state = reset()
        done = False
        while not done:
          act = pi[state, :]
          state, rew, done = env(state, act)
          total_reward += rew

      print(f'Q learning achieved: {total_reward / eval_episodes}')




    print("Starting Q-learning...")
    Q = np.zeros((K, num_of_actions))

    epoch_avg_rew = []
    epoch_rets = []
    epoch_progress = 0
    completed_epochs = 0
    episode_num = 0


    while completed_epochs < epochs:
        ep_ret = 0
        s = np.random.randint(0, K)

        while True:

            explore = np.random.binomial(1, e_t)
            if(explore == 1):
                a = np.random.randint(0, num_of_actions) #choose a random action to play at this round
            else:
                a = np.argmax(Q[s,:]) #pick the best action
            action = actions[a]
            next_state, reward, done = env(s, action)
            ep_ret += reward


            if done: #finish episode
                epoch_progress += 1
                epoch_rets.append(ep_ret) #store the total reward of the episode
                if epoch_progress == batch_size: #epoch finished
                    epoch_avg_rew.append(np.mean(epoch_rets))
                    epoch_rets = []
                    completed_epochs += 1
                    epoch_progress = 0

                break
            else:
                Q[s, a] += alpha*(reward + gamma*np.max(Q[next_state, :]) - Q[s, a])

            s = next_state


        episode_num += 1


    pi = generate_improved_policy(Q)
    plot_curves(epoch_avg_rew)

    if eval: eval_q_learning(pi)

    return epoch_avg_rew



##### FUNCTIONS #######

def plot_curves(pg_epoch_avg_ret):
  plt.figure()
  plt.title("Reward Per Episode vs. Episode")
  plt.xlabel("Episode")
  plt.ylabel("Reward per episode")
  plt.plot(np.arange(1,len(pg_epoch_avg_ret)+1), pg_epoch_avg_ret, color='b')
  plt.show()


def plot_pg_q_same_graph(pg_epoch_avg_ret, q_epoch_avg_ret):
  #Plot q learning and pg on the same graph
  plt.plot(np.arange(1,len(pg_epoch_avg_ret)+1), pg_epoch_avg_ret, label='PG')

  # Plot the second set of data and label it
  plt.plot(np.arange(1,len(pg_epoch_avg_ret)+1), q_epoch_avg_ret, label='Q')

  # Add a legend
  plt.legend()

  # Add labels and a title
  plt.xlabel('Epoch')
  plt.ylabel('Average episode return')
  plt.title('Average episode return vs Epoch')

  # Display the plot
  plt.show()


def reset():
    random_state = np.random.randint(0,K)
    return random_state
    # return one_hot_tensor(random_state)


def env(state, action):

    distr = np.ones(K) #the probability of each next_state (transition)
    quit = np.random.binomial(1, q) #if quit == 1 then terminal state.

    if(quit == 1):
        return 0, 0, True

    if(not all_relevant(state, action)): #at least one irrelevant video
        distr *= (1/K) #user chooses next video randomly from search bar
        next_state = np.random.choice(K, p = distr) #sample next state from the correct distribution
    else: # all relevant
        distr *= ((1-a)/K) #every video has a probability of (1-a)/K to be chosen
        for item in action:
            distr[item] += a/N #but those that are recommended also have an extra probability of a/N

        next_state = np.random.choice(K, p = distr) #sample next state from the correct distribution

    reward = is_cached(next_state) #if next_state is cached -> reward = 1 else 0

    return next_state, reward, False


def is_cached(item):
#     return C[item]
    return 1 if C[item] == 1 else 0

def all_relevant(s, w):
    for item in w:
        if(not U_bool[s, item]): #if at least one of recommendations is not relevant return False
            return False
    return True



##################EVALUATION#######################
def environment_model(state, action): #state is an integer, action is an N-tuple of recommendations
    scenarios = []
    if(all_relevant(state, action)):

        prob = (1-q)*(a/N + (1-a)/K) #probability to choose any recommended video
        for item in action:
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)

        prob = (1-q)*((1-a)/K) #probability to choose any not recommended video
        for item in range(K):
            if(item in action):
                continue
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)

    else:
        prob = (1-q)*((1-a)/K) #probability to choose any not recommended video
        for item in range(K):
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)

    t = (q, 0, 0, True) #terminal state, with probability q, terminate session
    scenarios.append(t)
    return scenarios



def generate_improved_policy(Q):
    new_pi = np.zeros((K, N), dtype=np.int8)
    # action_idxs = np.argmax(Q, axis=1)
    for s in range(K):
        best_action_idx = np.argmax(Q[s, :])
        new_pi[s, :] = actions[best_action_idx]
    return new_pi

def generate_random_policy():
    pi = np.zeros((K, N), dtype=np.int8)
    for s in range(K):
        action = np.random.choice(K, size=N, replace=False)
        pi[s, :] = action
    return pi



def policy_evaluation(pi, gamma = 1.0, epsilon = 1e-10):  #inputs: (1) policy to be evaluated, (2) model of the environment (transition probabilities, etc., see previous cell), (3) discount factor (with default = 1), (4) convergence error (default = 10^{-10})
    prev_V = np.zeros(K) # use as "cost-to-go", i.e. for V(s')
    while True: #performing iterations
        V = np.zeros(K) # current value function to be learnerd
        for s in range(K):  # do for every state
            for prob, next_state, reward, done in environment_model(s, pi[s, :]):  # calculate one Bellman step --> i.e., sum over all probabilities of transitions and reward for that state, the action suggested by the (fixed) policy, the reward earned (dictated by the model), and the cost-to-go from the next state (which is also decided by the model)
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < epsilon: #check if the new V estimate is close enough to the previous one;
            break # if yes, finish loop
        prev_V = V.copy() #freeze the new values (to be used as the next V(s'))
    return V


def policy_improvement(V, gamma=1.0):  # takes a value function (as the cost to go V(s')), a model, and a discount parameter
    Q = np.zeros((K, len(actions)), dtype=np.float64) #create a Q value array
    for s in range(K):        # for every state in the environment/model
        for i,a in enumerate(actions):  # and for every action in that state
            for prob, next_state, reward, done in environment_model(s, a):  #evaluate the action value based on the model and Value function given (which corresponds to the previous policy that we are trying to improve)
                Q[s][i] += prob * (reward + gamma * V[next_state] * (not done))
    new_pi = generate_improved_policy(Q)

    return Q, new_pi


def policy_iteration(gamma = 1.0, epsilon = 1e-10):
    t = 0
    pi = generate_random_policy()

    while True:
        old_pi = pi  #keep the old policy to compare with new
        V = policy_evaluation(pi, gamma, epsilon)   #evaluate latest policy --> you receive its converged value function
        Q, pi = policy_improvement(V,gamma)         #get a better policy using the value function of the previous one just calculated

        t += 1

        if(np.array_equal(old_pi, pi)):# you have converged to the optimal policy if the "improved" policy is exactly the same as in the previous step
            break

    return V,pi


def play_episodes(n, pi):
    total_reward = 0
    for i in range(n): #play n episodes
        state = np.random.randint(0,K)
        done = False
        while not done:
            action = pi[state, :]
            next_state, reward, done = env(state, action)
            total_reward += reward
            state = next_state

    avg_reward = total_reward / n
    print(f'Policy iteration acheived: {avg_reward}')




eval_episodes = 10000
epochs = 10
batch_size = 1000

# V,pi = policy_iteration(gamma=0.9)
# play_episodes(eval_episodes, pi)

pg_epoch_avg_ret = train(epochs=epochs, batch_size = batch_size, learning_rate=0.01, eval=True, eval_episodes = eval_episodes)
q_epoch_avg_ret = q_learning(gamma=0.9, epochs=epochs, batch_size=batch_size, eval=True, eval_episodes = eval_episodes)

plot_pg_q_same_graph(pg_epoch_avg_ret, q_epoch_avg_ret)




